# Open AIモデルのファインチューニング

このノートブックは、Open AIの[ファインチューニング](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst)ドキュメントで提供されている最新のガイダンスに基づいています。

ファインチューニングは、特定のユースケースやシナリオに関連する追加のデータとコンテキストで再学習させることにより、基盤モデルのパフォーマンスを向上させます。_few shot learning_や_retrieval augmented generation_のようなプロンプトエンジニアリング技術は、関連データでデフォルトのプロンプトを強化して品質を向上させることができます。ただし、これらのアプローチは対象となる基盤モデルの最大トークンウィンドウサイズによって制限されます。

ファインチューニングでは、必要なデータでモデル自体を効果的に再学習させ（最大トークンウィンドウに収まる以上の多くの例を使用可能にし）、推論時に例を提供する必要のない_カスタム_バージョンのモデルを展開します。これにより、プロンプト設計の効果が向上するだけでなく（トークンウィンドウを他の用途により柔軟に使える）、推論時にモデルに送るトークン数を減らすことでコストの削減も期待できます。

ファインチューニングには4つのステップがあります：
1. トレーニングデータを準備してアップロードする。
1. トレーニングジョブを実行してファインチューニング済みモデルを取得する。
1. ファインチューニング済みモデルを評価し、品質向上のために繰り返す。
1. 満足したらファインチューニング済みモデルを推論用に展開する。

すべての基盤モデルがファインチューニングをサポートしているわけではありません。最新情報は[OpenAIドキュメント](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst)を確認してください。また、以前にファインチューニングしたモデルをさらにファインチューニングすることも可能です。このチュートリアルでは、ファインチューニングの対象基盤モデルとして`gpt-35-turbo`を使用します。

---


### ステップ 1.1: データセットの準備

元素の周期表についての質問にリメリックで答えるチャットボットを作りましょう。この_シンプルな_チュートリアルでは、モデルをトレーニングするためのデータセットを、期待されるデータ形式を示すいくつかのサンプル応答例で作成します。実際のユースケースでは、はるかに多くの例を含むデータセットを作成する必要があります。もし存在すれば、オープンデータセット（あなたのアプリケーションドメイン向け）を利用し、ファインチューニング用に再フォーマットすることも可能です。

`gpt-35-turbo`に焦点を当て、シングルターンの応答（チャット完了）を求めているため、OpenAIのチャット完了要件を反映した[この推奨フォーマット](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst)を使って例を作成できます。マルチターンの会話コンテンツを想定する場合は、ファインチューニングプロセスで使用すべきメッセージを示す`weight`パラメータを含む[マルチターン例フォーマット](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst)を使用します。

ここではチュートリアルのためにより簡単なシングルターンフォーマットを使用します。データは[jsonlフォーマット](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst)で、1行に1レコードずつJSON形式のオブジェクトで表現されています。以下のスニペットは2レコードのサンプルを示しています。完全なサンプルセット（10例）は[training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl)をご覧ください。**注意：** 各レコードは必ず1行で定義しなければなりません（整形されたJSONファイルのように複数行に分割しないでください）。

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

実際のユースケースでは、良好な結果を得るためにはるかに多くの例が必要です。トレードオフは応答の質とファインチューニングにかかる時間・コストの間にあります。ここではプロセスを説明するためにファインチューニングを迅速に完了できるよう、小さなセットを使用しています。より複雑なファインチューニングチュートリアルについては、[このOpenAI Cookbookの例](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst)を参照してください。


---

### ステップ 1.2 データセットのアップロード

Files API を使ってデータをアップロードします。[こちらに記載されている通り](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file)です。コードを実行するには、以下の手順を事前に完了している必要があります：
 - `openai` Python パッケージをインストールしていること（最新機能を使うにはバージョン >=0.28.0 を使用してください）
 - `OPENAI_API_KEY` 環境変数に OpenAI API キーを設定していること
詳細は、コースで提供されている[セットアップガイド](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst)をご覧ください。

それでは、ローカルの JSONL ファイルからアップロード用のファイルを作成するコードを実行してください。


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### ステップ 2.1: SDK を使ってファインチューニングジョブを作成する


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### ステップ 2.2: ジョブのステータスを確認する

`client.fine_tuning.jobs` APIでできることは以下の通りです：
- `client.fine_tuning.jobs.list(limit=<n>)` - 最新のn件のファインチューニングジョブを一覧表示
- `client.fine_tuning.jobs.retrieve(<job_id>)` - 特定のファインチューニングジョブの詳細を取得
- `client.fine_tuning.jobs.cancel(<job_id>)` - ファインチューニングジョブをキャンセル
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - ジョブのイベントを最大n件一覧表示
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

プロセスの最初のステップは、_トレーニングファイルの検証_で、データが正しい形式であることを確認します。


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### ステップ 2.3: 進捗を監視するためにイベントを追跡する


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### ステップ 2.4: OpenAI ダッシュボードでステータスを確認する


OpenAIのウェブサイトにアクセスし、プラットフォームの_Fine-tuning_セクションを確認することで、ステータスを確認することもできます。ここでは現在のジョブのステータスが表示されるほか、過去のジョブ実行履歴を追跡することも可能です。このスクリーンショットでは、前回の実行が失敗し、2回目の実行が成功したことがわかります。背景として、最初の実行時にJSONファイルのレコードが誤った形式であったために失敗しましたが、修正後の2回目の実行は正常に完了し、モデルが使用可能になりました。

![Fine-tuning job status](../../../../../translated_images/ja/fine-tuned-model-status.563271727bf7bfba.png)


ステータスメッセージとメトリクスは、以下のようにビジュアルダッシュボードをさらにスクロールして表示することもできます:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/ja/fine-tuned-messages-panel.4ed0c2da5ea1313b.png) |  ![Metrics](../../../../../translated_images/ja/fine-tuned-metrics-panel.700d7e4995a65229.png)|


---

### ステップ 3.1: ID を取得し、コードでファインチューニング済みモデルをテストする


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### ステップ 3.2: Playgroundでファインチューニング済みモデルの読み込みとテスト

ファインチューニング済みモデルは、2つの方法でテストできます。まず、Playgroundにアクセスし、Modelsのドロップダウンから新しくファインチューニングしたモデルを選択する方法です。もう一つの方法は、ファインチューニングパネルに表示されている「Playground」オプション（上のスクリーンショット参照）を使うことで、基盤モデルとファインチューニング済みモデルのバージョンを並べて比較できるビューが起動します。

![Fine-tuning job status](../../../../../translated_images/ja/fine-tuned-playground-compare.56e06f0ad8922016.png)

トレーニングデータで使用したシステムコンテキストを入力し、テスト用の質問を提供してください。両側に同じコンテキストと質問が反映されるのがわかります。比較を実行すると、出力の違いが確認できます。_ファインチューニング済みモデルは、例で示したフォーマットで応答を生成するのに対し、基盤モデルは単にシステムプロンプトに従う点に注目してください_。

![Fine-tuning job status](../../../../../translated_images/ja/fine-tuned-playground-launch.5a26495c983c6350.png)

比較ビューでは、各モデルのトークン数と推論にかかった時間も表示されます。**この例はプロセスを示すための単純なものであり、実際のデータセットやシナリオを反映したものではありません**。両方のサンプルで同じトークン数（システムコンテキストとユーザープロンプトが同一）である一方、ファインチューニング済みモデルの方が推論に時間がかかる（カスタムモデル）ことがわかるでしょう。

実際のシナリオでは、このような単純な例ではなく、実データ（例：カスタマーサービス向けの製品カタログ）に対してファインチューニングを行い、応答の品質がより明確になります。_その_文脈では、基盤モデルで同等の応答品質を得るには、より高度なカスタムプロンプトエンジニアリングが必要となり、トークン使用量や推論処理時間が増加する可能性があります。_これを試すには、OpenAI Cookbookのファインチューニング例を参照してください。_

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責事項**：  
本書類はAI翻訳サービス「Co-op Translator」（https://github.com/Azure/co-op-translator）を使用して翻訳されました。正確性の向上に努めておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。原文の言語による文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用により生じた誤解や誤訳について、当方は一切の責任を負いかねます。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
